In [36]:
import pandas as pd
import numpy as np

In [28]:
df_admissions = pd.read_csv('./data/visitors_per_year_and_movie.csv')

In [29]:
# df_admissions['Directors'].fillna('Unknown', inplace=True)

In [30]:
df_admissions.isnull().sum()

Unnamed: 0           0
original title       0
producing country    0
production year      0
directors            0
visitors             0
dtype: int64

In [31]:
# Checked if values of the columns are identical
# (df_admissions['Admissions null-null'].str.replace(' ', '').astype(int) - df_admissions['Total EU27+GB since 1996'].str.replace(' ', '').astype(int)).sum()

In [32]:
# df_admissions_clean = df_admissions.drop('Total EU27+GB since 1996', axis=1)
df_admissions.rename(columns={'Admissions null-null': 'visitors'}, inplace=True)
df_admissions.columns = df_admissions.columns.str.lower()
# df_admissions['visitors'] = df_admissions['visitors'].str.replace(' ', '').astype('int64')

In [33]:
df_admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13669 entries, 0 to 13668
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   unnamed: 0         13669 non-null  int64 
 1   original title     13669 non-null  object
 2   producing country  13669 non-null  object
 3   production year    13669 non-null  int64 
 4   directors          13669 non-null  object
 5   visitors           13669 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 640.9+ KB


In [34]:
# df_admissions_clean.to_csv('./data/visitors_per_year_and_movie.csv', sep=',')

In [35]:
df_movie_list_years = pd.read_csv('./Data/movie_list_years.csv')
df_akas = pd.read_csv('./Data/title.akas.csv')
df_basics = pd.read_csv('./Data/title.basics.csv')
df_crew = pd.read_csv('./Data/title.crew.csv')
df_principals = pd.read_csv('./Data/title.principals.csv')
df_ratings = pd.read_csv('./Data/title.ratings.csv')
df_names = pd.read_csv('./Data/name.basics.csv')

In [38]:
df_movie_list_years.replace('\\N', np.nan)
df_akas.replace('\\N', np.nan)
df_basics.replace('\\N', np.nan)
df_crew.replace('\\N', np.nan)
df_principals.replace('\\N', np.nan)
df_ratings.replace('\\N', np.nan)
df_names.replace('\\N', np.nan)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
...,...,...,...,...,...,...
13779205,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13779206,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
13779207,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
13779208,nm9993718,Aayush Nair,NaN,NaN,cinematographer,tt8736744


In [44]:
df_akas.rename(columns={'titleId' : 'tconst'}, inplace=True)